In [1]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import optuna

In [2]:
# local
root_path = '../data/open/머신러닝'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

In [3]:
# 데이터 불러오기
#train_file = '201812_vif_drop_All'
#train_file = '201812_corr_drop_All'
train_file = '201812_processed_All'
train_df = pd.read_parquet(f'{root_path}/{train_file}.parquet')

test_file = f'{root_path}/test_12월_통합.parquet'
test_df = pd.read_parquet(test_file)

### 1단계 -> Segment == E vs Other

In [5]:
train_df["Segment1"] = train_df["Segment"].apply(lambda x: "E" if x == "E" else "other")

In [6]:
exclude_cols = ['ID', '기준년월', 'Segment', 'Segment1']
feature_cols = [
    col for col in train_df.columns
    if col not in exclude_cols
]

In [7]:
# 학습용 피처/타겟 정의
X = train_df[feature_cols]
y = train_df['Segment1']

In [8]:
# X와 test_df 양쪽에 모두 존재하는 컬럼만 선택
common_cols = [col for col in X.columns if col in test_df.columns]
X = X[common_cols]
X_test = test_df[common_cols]

In [9]:
# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
def objective(trial):
    params = {
        # boosting 반복 횟수
        "iterations": trial.suggest_int("iterations", 300, 1500),
        # 한번에 얼마나 빠르게 학습하지지
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        # 트리의 깊이, 깊을수록 복잡한 모델이지만 과적합 가능성 O
        "depth": trial.suggest_int("depth", 4, 10), # --> 튜닝
        # L2 정규화 , 높을수록 규제 강화 - 과적합 억제
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        # 수치형 피쳐를 이산화 할 때 사용하는 경계 개수 -> 수치형 많으면 영향력 있음
        "border_count": trial.suggest_int("border_count", 32, 255),
        # 트리 분기시 f.i 계산에 노이즈를 얼마나 줄 것인가
        "random_strength": trial.suggest_float("random_strength", 0.1, 1),
        # 샘플링의 다양성 조절 -> 과소적합일 땐 높이고, 과적합일 땐 낮게
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        # 손실 함수 : 다중분류
        "loss_function": "Logloss",
        # 평가 지표
        "eval_metric": "F1",
        # 학습 로그를 몇 step마다 보여줄지
        "verbose": 100,
        # 트리 분기 시 feature 중요도 계산에 주는 노이즈 크기
        "random_state": 42
    }
    # early_stopping_rounds - 지정된 라운드 동안 성능 개선이 되지 않으면 학습 중지
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=100)

    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='micro')
    return f1

In [11]:
# Optuna 튜닝 시작
study = optuna.create_study(direction="maximize")
# XGB 때 n_trials=30 으로 했었는데 너무 오래걸려서 일단 20으로 설정
# n_trials는 optuna가 시도할 하이퍼파라미터 조합의 개수 (몇번 학습할건지)
study.optimize(objective, n_trials=30)

[I 2025-07-11 00:22:32,116] A new study created in memory with name: no-name-b35023e5-8da3-4f1f-af84-669bcbd4ded9


0:	learn: 0.7338074	test: 0.7322357	best: 0.7322357 (0)	total: 665ms	remaining: 6m 22s
100:	learn: 0.8022430	test: 0.7969516	best: 0.7970707 (99)	total: 48.3s	remaining: 3m 47s
200:	learn: 0.8283855	test: 0.8177830	best: 0.8177830 (200)	total: 1m 35s	remaining: 2m 57s
300:	learn: 0.8445309	test: 0.8274312	best: 0.8278536 (297)	total: 2m 19s	remaining: 2m 7s
400:	learn: 0.8568263	test: 0.8338149	best: 0.8340475 (399)	total: 3m 4s	remaining: 1m 20s
500:	learn: 0.8663556	test: 0.8384693	best: 0.8386637 (498)	total: 3m 47s	remaining: 34.1s
575:	learn: 0.8727166	test: 0.8395038	best: 0.8402436 (549)	total: 4m 20s	remaining: 0us

bestTest = 0.8402435864
bestIteration = 549

Shrink model to first 550 iterations.


[I 2025-07-11 00:26:59,143] Trial 0 finished with value: 0.93835 and parameters: {'iterations': 576, 'learning_rate': 0.07343449822671735, 'depth': 8, 'l2_leaf_reg': 5.248081733627526, 'border_count': 43, 'random_strength': 0.6668615616405228, 'bagging_temperature': 0.06183497026941365}. Best is trial 0 with value: 0.93835.


0:	learn: 0.7338907	test: 0.7316637	best: 0.7316637 (0)	total: 490ms	remaining: 2m 57s
100:	learn: 0.8092938	test: 0.8039358	best: 0.8040005 (99)	total: 48.6s	remaining: 2m 6s
200:	learn: 0.8355399	test: 0.8233833	best: 0.8233833 (200)	total: 1m 33s	remaining: 1m 15s
300:	learn: 0.8509688	test: 0.8311215	best: 0.8311215 (300)	total: 2m 18s	remaining: 28.5s
362:	learn: 0.8593343	test: 0.8362539	best: 0.8364321 (360)	total: 2m 46s	remaining: 0us

bestTest = 0.8364320711
bestIteration = 360

Shrink model to first 361 iterations.


[I 2025-07-11 00:29:52,077] Trial 1 finished with value: 0.9369875 and parameters: {'iterations': 363, 'learning_rate': 0.08797671703200764, 'depth': 8, 'l2_leaf_reg': 9.666895970015988, 'border_count': 45, 'random_strength': 0.6372513111077474, 'bagging_temperature': 0.13013473944435616}. Best is trial 0 with value: 0.93835.


...

[I 2025-07-11 05:54:28,868] Trial 27 finished with value: 0.94135 and parameters: {'iterations': 787, 'learning_rate': 0.08217912899111399, 'depth': 8, 'l2_leaf_reg': 9.483184723148232, 'border_count': 159, 'random_strength': 0.742766608249526, 'bagging_temperature': 0.6965108826235061}. Best is trial 26 with value: 0.94315.


0:	learn: 0.7371006	test: 0.7333513	best: 0.7333513 (0)	total: 671ms	remaining: 12m 1s
100:	learn: 0.8213600	test: 0.8102638	best: 0.8102638 (100)	total: 1m 8s	remaining: 11m 4s
200:	learn: 0.8505055	test: 0.8279874	best: 0.8280905 (199)	total: 2m 14s	remaining: 9m 44s
300:	learn: 0.8689499	test: 0.8367981	best: 0.8369819 (298)	total: 3m 17s	remaining: 8m 28s
400:	learn: 0.8828897	test: 0.8411590	best: 0.8412678 (398)	total: 4m 19s	remaining: 7m 17s
500:	learn: 0.8947381	test: 0.8448593	best: 0.8448593 (500)	total: 5m 22s	remaining: 6m 9s
600:	learn: 0.9048330	test: 0.8470869	best: 0.8473603 (597)	total: 6m 24s	remaining: 5m 3s
700:	learn: 0.9151616	test: 0.8495324	best: 0.8496808 (696)	total: 7m 26s	remaining: 3m 58s
800:	learn: 0.9231649	test: 0.8510419	best: 0.8511887 (799)	total: 8m 28s	remaining: 2m 54s
900:	learn: 0.9308868	test: 0.8524125	best: 0.8525139 (892)	total: 9m 30s	remaining: 1m 50s
1000:	learn: 0.9375488	test: 0.8537929	best: 0.8539123 (982)	total: 10m 32s	remaining: 4

[I 2025-07-11 06:05:55,790] Trial 28 finished with value: 0.9434625 and parameters: {'iterations': 1076, 'learning_rate': 0.0984473970334064, 'depth': 9, 'l2_leaf_reg': 9.308556840752988, 'border_count': 113, 'random_strength': 0.5988105034815305, 'bagging_temperature': 0.8720065043629223}. Best is trial 28 with value: 0.9434625.


0:	learn: 0.7297047	test: 0.7286666	best: 0.7286666 (0)	total: 1.67s	remaining: 29m 6s
100:	learn: 0.8298424	test: 0.8124035	best: 0.8124035 (100)	total: 2m 39s	remaining: 24m 56s
200:	learn: 0.8634061	test: 0.8298966	best: 0.8298966 (200)	total: 5m 18s	remaining: 22m 22s
300:	learn: 0.8854273	test: 0.8378107	best: 0.8378545 (299)	total: 7m 57s	remaining: 19m 45s
400:	learn: 0.9038076	test: 0.8421665	best: 0.8422209 (391)	total: 10m 37s	remaining: 17m 7s
500:	learn: 0.9180508	test: 0.8457666	best: 0.8457666 (500)	total: 13m 15s	remaining: 14m 28s
600:	learn: 0.9306524	test: 0.8473309	best: 0.8478737 (592)	total: 15m 54s	remaining: 11m 49s
700:	learn: 0.9401345	test: 0.8492048	best: 0.8492207 (694)	total: 18m 33s	remaining: 9m 11s
800:	learn: 0.9490869	test: 0.8508572	best: 0.8510296 (792)	total: 21m 12s	remaining: 6m 32s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8512678594
bestIteration = 820

Shrink model to first 821 iterations.


[I 2025-07-11 06:29:12,654] Trial 29 finished with value: 0.942225 and parameters: {'iterations': 1048, 'learning_rate': 0.09844228913173886, 'depth': 10, 'l2_leaf_reg': 9.293029609793729, 'border_count': 32, 'random_strength': 0.5955894644404489, 'bagging_temperature': 0.9847601273731006}. Best is trial 28 with value: 0.9434625.


In [12]:
# 이렇게 하면 총 10번 추가로 돌아감
# study.optimize(objective, n_trials=10)

In [13]:
# 결과 출력 (궁금하니까 --> 여기선 F1이 micro가 아니라 낮을 수 밖에 없다.)
print("Best F1 score:", study.best_value)
print("Best parameters:", study.best_params)

Best F1 score: 0.9434625
Best parameters: {'iterations': 1076, 'learning_rate': 0.0984473970334064, 'depth': 9, 'l2_leaf_reg': 9.308556840752988, 'border_count': 113, 'random_strength': 0.5988105034815305, 'bagging_temperature': 0.8720065043629223}


In [14]:
# 최적 파라미터로 모델 학습
# Optuna가 튜닝한 것이 아닌 것들은 다시 넣어줘야함.
# Optuna에 넣은 이유는 기본적으로 들어있어야 하기 때문에
best_params = study.best_params
best_params.update({
    "loss_function": "Logloss",
    "eval_metric": "F1",
    "random_state": 42,
    "verbose": 100
})

In [15]:
# 실제 모델
model = CatBoostClassifier(**best_params)
model.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          early_stopping_rounds=50)

0:	learn: 0.7371006	test: 0.7333513	best: 0.7333513 (0)	total: 727ms	remaining: 13m 1s
100:	learn: 0.8213600	test: 0.8102638	best: 0.8102638 (100)	total: 1m 11s	remaining: 11m 33s
200:	learn: 0.8505055	test: 0.8279874	best: 0.8280905 (199)	total: 2m 19s	remaining: 10m 8s
300:	learn: 0.8689499	test: 0.8367981	best: 0.8369819 (298)	total: 3m 25s	remaining: 8m 49s
400:	learn: 0.8828897	test: 0.8411590	best: 0.8412678 (398)	total: 4m 30s	remaining: 7m 34s
500:	learn: 0.8947381	test: 0.8448593	best: 0.8448593 (500)	total: 5m 34s	remaining: 6m 24s
600:	learn: 0.9048330	test: 0.8470869	best: 0.8473603 (597)	total: 6m 39s	remaining: 5m 15s
700:	learn: 0.9151616	test: 0.8495324	best: 0.8496808 (696)	total: 7m 44s	remaining: 4m 8s
800:	learn: 0.9231649	test: 0.8510419	best: 0.8511887 (799)	total: 8m 48s	remaining: 3m 1s
900:	learn: 0.9308868	test: 0.8524125	best: 0.8525139 (892)	total: 9m 52s	remaining: 1m 55s
1000:	learn: 0.9375488	test: 0.8537929	best: 0.8539123 (982)	total: 10m 56s	remaining:

In [16]:
# 예측
test_df['Segment_pred'] = model.predict(X_test)

In [17]:
test_df['Segment_pred'].value_counts()

Segment_pred
E        81115
other    18885
Name: count, dtype: int64

### 2단계 -> Segment == C or D vs Other (E 제거)

In [19]:
train_df2= train_df[train_df['Segment'] != 'E'].copy()
train_df2['Segment1'] = train_df2['Segment'].apply(lambda x: x if x in ['C', 'D'] else 'other')

In [20]:
exclude_cols = ['ID', '기준년월', 'Segment', 'Segment1']
feature_cols = [
    col for col in train_df.columns
    if col not in exclude_cols
]

In [21]:
# 학습용 피처/타겟 정의
X2 = train_df2[feature_cols]
y2 = train_df2['Segment1']

In [22]:
# 예측할 test subset (1단계 결과가 Other인 경우)
test_df2 = test_df[test_df['Segment_pred'] == 'other'].copy()

In [23]:
# X와 test_df 양쪽에 모두 존재하는 컬럼만 선택
common_cols = [col for col in X2.columns if col in test_df2.columns]
X2 = X2[common_cols]
X_test2 = test_df2[common_cols]

In [24]:
# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X2, y2, test_size=0.2, random_state=42, stratify=y2)

In [25]:
def objective2(trial):
    params = {
        # boosting 반복 횟수
        "iterations": trial.suggest_int("iterations", 300, 1000),
        # 한번에 얼마나 빠르게 학습하지지
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        # 트리의 깊이, 깊을수록 복잡한 모델이지만 과적합 가능성 O
        "depth": trial.suggest_int("depth", 4, 8), # --> 튜닝
        # L2 정규화 , 높을수록 규제 강화 - 과적합 억제
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 3, 10),
        # 수치형 피쳐를 이산화 할 때 사용하는 경계 개수 -> 수치형 많으면 영향력 있음
        "border_count": trial.suggest_int("border_count", 32, 128),
        # 트리 분기시 f.i 계산에 노이즈를 얼마나 줄 것인가
        "random_strength": trial.suggest_float("random_strength", 0.5, 1.0),
        # 샘플링의 다양성 조절 -> 과소적합일 땐 높이고, 과적합일 땐 낮게
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.2, 1.0),
        # 손실 함수 : 다중분류
        "loss_function": "MultiClass",
        # 평가 지표
        "eval_metric": "TotalF1:average=Micro",
        # 학습 로그를 몇 step마다 보여줄지
        "verbose": 100,
        # 트리 분기 시 feature 중요도 계산에 주는 노이즈 크기
        "random_state": 42
    }
    # early_stopping_rounds - 지정된 라운드 동안 성능 개선이 되지 않으면 학습 중지
    model2 = CatBoostClassifier(**params)
    model2.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=30, verbose=100)

    preds = model2.predict(X_val)
    f1 = f1_score(y_val, preds, average='micro')
    return f1

In [26]:
# Optuna 튜닝 시작
study2 = optuna.create_study(direction="maximize")
# XGB 때 n_trials=30 으로 했었는데 너무 오래걸려서 일단 20으로 설정
# n_trials는 optuna가 시도할 하이퍼파라미터 조합의 개수 (몇번 학습할건지)
study2.optimize(objective2, n_trials=20)

[I 2025-07-11 06:41:15,805] A new study created in memory with name: no-name-b2a2d898-330d-4357-b688-1000bbae27c2


0:	learn: 0.7978376	test: 0.7929952	best: 0.7929952 (0)	total: 239ms	remaining: 2m 50s
100:	learn: 0.8406773	test: 0.8294627	best: 0.8296510 (97)	total: 24.3s	remaining: 2m 27s
200:	learn: 0.8514421	test: 0.8400075	best: 0.8400703 (199)	total: 47.1s	remaining: 2m
300:	learn: 0.8587861	test: 0.8437108	best: 0.8440246 (280)	total: 1m 8s	remaining: 1m 34s
400:	learn: 0.8637448	test: 0.8467863	best: 0.8469119 (383)	total: 1m 30s	remaining: 1m 10s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8481672106
bestIteration = 430

Shrink model to first 431 iterations.


[I 2025-07-11 06:43:01,768] Trial 0 finished with value: 0.8481672106452423 and parameters: {'iterations': 715, 'learning_rate': 0.08977742666096838, 'depth': 5, 'l2_leaf_reg': 8.189775090377125, 'border_count': 57, 'random_strength': 0.9259953841251828, 'bagging_temperature': 0.8084696479337616}. Best is trial 0 with value: 0.8481672106452423.


0:	learn: 0.8140790	test: 0.8111348	best: 0.8111348 (0)	total: 409ms	remaining: 4m 49s
100:	learn: 0.8492923	test: 0.8374969	best: 0.8376224 (99)	total: 38.9s	remaining: 3m 54s
200:	learn: 0.8643254	test: 0.8444012	best: 0.8447150 (198)	total: 1m 15s	remaining: 3m 12s
300:	learn: 0.8755924	test: 0.8489204	best: 0.8495481 (286)	total: 1m 52s	remaining: 2m 32s
400:	learn: 0.8841132	test: 0.8501130	best: 0.8509289 (385)	total: 2m 27s	remaining: 1m 53s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.850928948
bestIteration = 385

Shrink model to first 386 iterations.


[I 2025-07-11 06:45:37,644] Trial 1 finished with value: 0.8509289480291238 and parameters: {'iterations': 710, 'learning_rate': 0.0877798065742581, 'depth': 7, 'l2_leaf_reg': 3.148923514640135, 'border_count': 107, 'random_strength': 0.5119786600262362, 'bagging_temperature': 0.5256113380838991}. Best is trial 1 with value: 0.8509289480291238.


...

[I 2025-07-11 07:49:33,473] Trial 17 finished with value: 0.8490459452673864 and parameters: {'iterations': 997, 'learning_rate': 0.07073571423178805, 'depth': 7, 'l2_leaf_reg': 4.149174250176879, 'border_count': 48, 'random_strength': 0.8666505004882072, 'bagging_temperature': 0.9039596686099076}. Best is trial 10 with value: 0.8523725834797891.


0:	learn: 0.8027807	test: 0.7990836	best: 0.7990836 (0)	total: 1.01s	remaining: 9m 34s
100:	learn: 0.8427957	test: 0.8304670	best: 0.8306553 (95)	total: 1m 44s	remaining: 8m 2s
200:	learn: 0.8541098	test: 0.8385011	best: 0.8385011 (197)	total: 3m 27s	remaining: 6m 19s
300:	learn: 0.8639802	test: 0.8442129	best: 0.8442129 (300)	total: 5m 11s	remaining: 4m 37s
400:	learn: 0.8725324	test: 0.8467863	best: 0.8467863 (400)	total: 6m 55s	remaining: 2m 54s
500:	learn: 0.8790447	test: 0.8487949	best: 0.8487949 (495)	total: 8m 39s	remaining: 1m 10s
568:	learn: 0.8831560	test: 0.8504896	best: 0.8504896 (562)	total: 9m 50s	remaining: 0us

bestTest = 0.8504895807
bestIteration = 562

Shrink model to first 563 iterations.


[I 2025-07-11 07:59:27,605] Trial 18 finished with value: 0.8504895807180517 and parameters: {'iterations': 569, 'learning_rate': 0.05699262388580066, 'depth': 8, 'l2_leaf_reg': 6.10363543039273, 'border_count': 70, 'random_strength': 0.9459055555294112, 'bagging_temperature': 0.9088135947447338}. Best is trial 10 with value: 0.8523725834797891.


0:	learn: 0.8075197	test: 0.8067411	best: 0.8067411 (0)	total: 344ms	remaining: 4m 48s
100:	learn: 0.8465618	test: 0.8361160	best: 0.8361788 (99)	total: 30.3s	remaining: 3m 41s
200:	learn: 0.8592254	test: 0.8437108	best: 0.8441501 (199)	total: 58s	remaining: 3m 4s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8462842079
bestIteration = 233

Shrink model to first 234 iterations.


[I 2025-07-11 08:00:44,578] Trial 19 finished with value: 0.8462842078835049 and parameters: {'iterations': 840, 'learning_rate': 0.09356236785176296, 'depth': 6, 'l2_leaf_reg': 3.9175662254796304, 'border_count': 109, 'random_strength': 0.5757656522364232, 'bagging_temperature': 0.5150104761060543}. Best is trial 10 with value: 0.8523725834797891.


In [27]:
# 결과 출력 (궁금하니까 --> 여기선 F1이 micro가 아니라 낮을 수 밖에 없다.)
print("Best F1 score:", study2.best_value)
print("Best parameters:", study2.best_params)

Best F1 score: 0.8523725834797891
Best parameters: {'iterations': 853, 'learning_rate': 0.0989412064563703, 'depth': 8, 'l2_leaf_reg': 4.7377949762190195, 'border_count': 79, 'random_strength': 0.9723122632847294, 'bagging_temperature': 0.9563408727257483}


In [28]:
# 최적 파라미터로 모델 학습
# Optuna가 튜닝한 것이 아닌 것들은 다시 넣어줘야함.
# Optuna에 넣은 이유는 기본적으로 들어있어야 하기 때문에
best_params2 = study2.best_params
best_params2.update({
    "loss_function": "MultiClass",
    "eval_metric": "TotalF1:average=Micro",
    "random_state": 42,
    "verbose": 100
})

In [29]:
# 실제 모델
model2 = CatBoostClassifier(**best_params2)
model2.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          early_stopping_rounds=30)

0:	learn: 0.7991715	test: 0.7936857	best: 0.7936857 (0)	total: 1.04s	remaining: 14m 48s
100:	learn: 0.8523523	test: 0.8364926	best: 0.8365554 (99)	total: 1m 46s	remaining: 13m 14s
200:	learn: 0.8686564	test: 0.8441501	best: 0.8442757 (198)	total: 3m 32s	remaining: 11m 30s
300:	learn: 0.8801902	test: 0.8489204	best: 0.8489204 (300)	total: 5m 19s	remaining: 9m 45s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8523725835
bestIteration = 366

Shrink model to first 367 iterations.


In [30]:
# 예측
test_df2['Segment_pred'] = model2.predict(X_test2).flatten()

In [31]:
test_df2['Segment_pred'].value_counts()

Segment_pred
D        14500
C         4377
other        8
Name: count, dtype: int64

In [32]:
# 예측 결과 병합
# 중복된 ID 제거: 마지막 값을 기준으로 유지
test_df2_dedup = test_df2.drop_duplicates(subset='ID', keep='last')
test_df2_indexed = test_df2_dedup.set_index('ID')

# test_df의 Segment_pred를 ID 기준으로 업데이트
test_df.loc[test_df['ID'].isin(test_df2_indexed.index), 'Segment_pred'] = \
    test_df.loc[test_df['ID'].isin(test_df2_indexed.index), 'ID'].map(test_df2_indexed['Segment_pred'])

### 3단계 -> Segment == a or B vs Other (C, D, E 제거)

In [34]:
# 학습용 피처/타겟 정의
train_df3 = train_df[train_df['Segment'].isin(['A', 'B'])].copy()
train_df3['Segment1'] = train_df3['Segment']

In [35]:
exclude_cols = ['ID', '기준년월', 'Segment', 'Segment1']
feature_cols = [
    col for col in train_df.columns
    if col not in exclude_cols
]

In [36]:
# 학습용 피처/타겟 정의
X3 = train_df3[feature_cols]
y3 = train_df3['Segment1']

In [37]:
# 예측할 test subset (1단계 결과가 Other인 경우)
test_df3 = test_df[test_df['Segment_pred'] == 'other'].copy()

In [38]:
# X와 test_df 양쪽에 모두 존재하는 컬럼만 선택
common_cols = [col for col in X3.columns if col in test_df3.columns]
X3 = X3[common_cols]
X_test3 = test_df3[common_cols]

In [39]:
# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X3, y3, test_size=0.2, random_state=42, stratify=y3)

In [40]:
def objective3(trial):
    params = {
        # boosting 반복 횟수
        "iterations": trial.suggest_int("iterations", 100, 500),
        # 한번에 얼마나 빠르게 학습하지지
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.2),
        # 트리의 깊이, 깊을수록 복잡한 모델이지만 과적합 가능성 O
        "depth": trial.suggest_int("depth", 3, 6), # --> 튜닝
        # L2 정규화 , 높을수록 규제 강화 - 과적합 억제
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 5, 20),
        # 수치형 피쳐를 이산화 할 때 사용하는 경계 개수 -> 수치형 많으면 영향력 있음
        "border_count": trial.suggest_int("border_count", 32, 64),
        # 트리 분기시 f.i 계산에 노이즈를 얼마나 줄 것인가
        "random_strength": trial.suggest_float("random_strength", 0.8, 2.0),
        # 샘플링의 다양성 조절 -> 과소적합일 땐 높이고, 과적합일 땐 낮게
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.5, 1.0),
        # 손실 함수 : 다중분류
        "loss_function": "MultiClass",
        # 평가 지표
        "eval_metric": "TotalF1:average=Micro",
        # 학습 로그를 몇 step마다 보여줄지
        "verbose": 100,
        # 트리 분기 시 feature 중요도 계산에 주는 노이즈 크기
        "random_state": 42
    }
    # early_stopping_rounds - 지정된 라운드 동안 성능 개선이 되지 않으면 학습 중지
    model3 = CatBoostClassifier(**params)
    model3.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=20, verbose=100)

    preds = model3.predict(X_val)
    f1 = f1_score(y_val, preds, average='micro')
    return f1

In [93]:
# Optuna 튜닝 시작
study3 = optuna.create_study(direction="maximize")
# XGB 때 n_trials=30 으로 했었는데 너무 오래걸려서 일단 20으로 설정
# n_trials는 optuna가 시도할 하이퍼파라미터 조합의 개수 (몇번 학습할건지)
study3.optimize(objective3, n_trials=15)

[I 2025-07-11 09:08:20,375] A new study created in memory with name: no-name-c46557a9-8d40-4724-ba7e-f3978ee92c82


0:	learn: 0.9256757	test: 0.8421053	best: 0.8421053 (0)	total: 20.1ms	remaining: 6.32s


[I 2025-07-11 09:08:21,035] Trial 0 finished with value: 0.9210526315789473 and parameters: {'iterations': 315, 'learning_rate': 0.16773821836168157, 'depth': 4, 'l2_leaf_reg': 11.91497712793132, 'border_count': 53, 'random_strength': 0.9685255061348419, 'bagging_temperature': 0.7931004119765275}. Best is trial 0 with value: 0.9210526315789473.


Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9210526316
bestIteration = 14

Shrink model to first 15 iterations.
0:	learn: 0.8918919	test: 0.8684211	best: 0.8684211 (0)	total: 52ms	remaining: 10.3s


[I 2025-07-11 09:08:22,627] Trial 1 finished with value: 0.9210526315789473 and parameters: {'iterations': 199, 'learning_rate': 0.11977775316246801, 'depth': 6, 'l2_leaf_reg': 13.108798003183214, 'border_count': 61, 'random_strength': 1.2301980602593225, 'bagging_temperature': 0.7861013058478769}. Best is trial 0 with value: 0.9210526315789473.


...

Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9473684211
bestIteration = 2

Shrink model to first 3 iterations.
0:	learn: 0.8986486	test: 0.7894737	best: 0.7894737 (0)	total: 40.4ms	remaining: 4.12s


[I 2025-07-11 09:08:32,455] Trial 13 finished with value: 0.9210526315789473 and parameters: {'iterations': 103, 'learning_rate': 0.1507835209736169, 'depth': 5, 'l2_leaf_reg': 18.36661949396114, 'border_count': 42, 'random_strength': 0.8026972351573419, 'bagging_temperature': 0.969876843625368}. Best is trial 12 with value: 0.9473684210526315.


Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9210526316
bestIteration = 13

Shrink model to first 14 iterations.
0:	learn: 0.8918919	test: 0.8421053	best: 0.8421053 (0)	total: 39.5ms	remaining: 14.8s


[I 2025-07-11 09:08:33,356] Trial 14 finished with value: 0.9210526315789473 and parameters: {'iterations': 376, 'learning_rate': 0.1995193853579867, 'depth': 4, 'l2_leaf_reg': 16.61208082802828, 'border_count': 56, 'random_strength': 0.9588124692778636, 'bagging_temperature': 0.9235807267281838}. Best is trial 12 with value: 0.9473684210526315.


Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9210526316
bestIteration = 13

Shrink model to first 14 iterations.


In [95]:
# 결과 출력 (궁금하니까 --> 여기선 F1이 micro가 아니라 낮을 수 밖에 없다.)
print("Best F1 score:", study3.best_value)
print("Best parameters:", study3.best_params)

Best F1 score: 0.9473684210526315
Best parameters: {'iterations': 230, 'learning_rate': 0.1315727592190663, 'depth': 5, 'l2_leaf_reg': 17.83470393444972, 'border_count': 63, 'random_strength': 0.8035538516258349, 'bagging_temperature': 0.9534560110981747}


In [99]:
# 최적 파라미터로 모델 학습
# Optuna가 튜닝한 것이 아닌 것들은 다시 넣어줘야함.
# Optuna에 넣은 이유는 기본적으로 들어있어야 하기 때문에
best_params3 = study3.best_params
best_params3.update({
    "loss_function": "MultiClass",
    "eval_metric": "TotalF1:average=Micro",
    "random_state": 42,
    "verbose": 100
})

In [101]:
# 실제 모델
model3 = CatBoostClassifier(**best_params3)
model3.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          early_stopping_rounds=20)

0:	learn: 0.9256757	test: 0.8947368	best: 0.8947368 (0)	total: 50.4ms	remaining: 11.5s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9473684211
bestIteration = 2

Shrink model to first 3 iterations.


In [103]:
# 예측
test_df3['Segment_pred'] = model3.predict(X_test3).flatten()

In [105]:
test_df3['Segment_pred'].value_counts()

Segment_pred
A    8
Name: count, dtype: int64

In [107]:
# 예측 결과 병합
# 중복된 ID 제거: 마지막 값을 기준으로 유지
test_df3_dedup = test_df3.drop_duplicates(subset='ID', keep='last')
test_df3_indexed = test_df3_dedup.set_index('ID')

# test_df의 Segment_pred를 ID 기준으로 업데이트
test_df.loc[test_df['ID'].isin(test_df3_indexed.index), 'Segment_pred'] = \
    test_df.loc[test_df['ID'].isin(test_df3_indexed.index), 'ID'].map(test_df3_indexed['Segment_pred'])

# 최종 예측 결과 저장
submission = test_df[['ID', 'Segment_pred']].copy()
submission.rename(columns={'Segment_pred': 'Segment'}, inplace=True)
submission.to_parquet(f'../results/{train_file}_catboost_model3_submission4.parquet', index=False)
print(f"{train_file}_catboost_model3_submission 저장 완료!")

201812_processed_All_catboost_model3_submission 저장 완료!
